In [69]:
from collections import Counter
from datetime import datetime
 
import json
 
from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
 
import numpy as np

In [70]:
t1 = datetime.now()
with open("yelp_data/dataset/review.json") as f:
    reviews = f.read().strip().split("\n")
reviews = [json.loads(review) for review in reviews]
print(datetime.now() - t1)

0:01:08.040517


In [71]:
print(len(reviews))

5261669


In [105]:
texts = [review['text'] for review in reviews]
 
# Convert our 5 classes into 2 (negative or positive)
numStars = [review['stars'] for review in reviews]
balanced_texts = []
balanced_labels = []
limit = 20000  # Change this to grow/shrink the dataset
pos_counts = [0, 0, 0, 0, 0]
for i in range(len(texts)):
    score = numStars[i]-1
    if pos_counts[score] < limit:
        balanced_texts.append(texts[i])
        balanced_labels.append(numStars[i]-1)
        pos_counts[score] += 1

In [106]:
print(balanced_labels)

[4, 4, 4, 4, 3, 3, 4, 3, 3, 2, 4, 3, 2, 0, 2, 4, 3, 0, 2, 2, 0, 2, 3, 2, 0, 2, 2, 1, 4, 2, 3, 3, 0, 3, 3, 3, 4, 3, 3, 3, 3, 1, 3, 3, 0, 0, 3, 3, 3, 3, 3, 2, 3, 3, 1, 2, 3, 3, 3, 1, 3, 0, 0, 2, 2, 1, 4, 3, 3, 2, 2, 2, 3, 3, 0, 3, 1, 2, 3, 2, 2, 3, 0, 3, 3, 3, 3, 4, 0, 3, 3, 3, 4, 4, 0, 0, 0, 0, 3, 0, 1, 3, 3, 2, 1, 0, 1, 2, 2, 0, 1, 3, 1, 2, 1, 3, 3, 1, 0, 4, 3, 3, 4, 0, 4, 3, 0, 1, 3, 3, 1, 2, 2, 4, 3, 3, 2, 0, 3, 3, 2, 2, 3, 3, 0, 1, 3, 4, 3, 1, 3, 3, 2, 3, 2, 1, 2, 3, 1, 1, 3, 2, 1, 2, 1, 2, 3, 1, 4, 2, 3, 1, 0, 3, 3, 4, 3, 3, 4, 4, 4, 4, 2, 3, 3, 1, 3, 3, 4, 3, 2, 3, 2, 4, 3, 1, 1, 4, 3, 0, 3, 3, 2, 4, 2, 4, 2, 3, 0, 4, 1, 1, 4, 3, 4, 1, 3, 3, 3, 2, 3, 3, 2, 4, 2, 0, 4, 3, 2, 1, 4, 3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 1, 2, 1, 2, 3, 3, 3, 4, 3, 3, 3, 3, 3, 2, 3, 2, 2, 4, 1, 1, 3, 2, 4, 4, 2, 3, 4, 3, 3, 2, 3, 3, 2, 2, 2, 2, 4, 4, 4, 2, 2, 4, 2, 3, 2, 4, 3, 3, 3, 3, 2, 2, 0, 2, 3, 2, 2, 4, 0, 3, 1, 2, 4, 4, 2, 0, 2, 2, 2, 3, 1, 1, 3, 2, 4, 1, 2, 2, 1, 3, 2, 1, 2, 3, 3, 1, 2, 4, 2, 2, 0, 3, 

In [93]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(balanced_texts)
sequences = tokenizer.texts_to_sequences(balanced_texts)
data = pad_sequences(sequences, maxlen=300)

In [114]:
from keras.utils import to_categorical

model = Sequential()
model.add(Embedding(20000, 128, input_length=300))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'],)
newLabels=np.array(balanced_labels)
newLabels=to_categorical(newLabels)
print(newLabels.shape)

(100000, 5)


In [115]:
history=model.fit(data, newLabels, validation_split=0.2, epochs=3)

Train on 80000 samples, validate on 20000 samples
Epoch 1/3
80000/80000 [==============================] - 995s 12ms/step - loss: 1.0969 - acc: 0.5273 - val_loss: 1.2112 - val_acc: 0.4374
Epoch 2/3
80000/80000 [==============================] - 990s 12ms/step - loss: 0.9052 - acc: 0.6143 - val_loss: 1.1451 - val_acc: 0.4763
Epoch 3/3
80000/80000 [==============================] - 990s 12ms/step - loss: 0.8322 - acc: 0.6455 - val_loss: 1.2221 - val_acc: 0.4314
